In [85]:
using CPUTime
using Polynomials
using Plots
using SpecialFunctions
using LinearAlgebra


In [86]:

Vector2D{T<:Real}=NamedTuple{(:x, :y), Tuple{T,T}}

Base. +(A::Vector2D{T},B::Vector2D{T}) where T = Vector2D{T}(Tuple(A).+Tuple(B))

Base. -(A::Vector2D{T}, B::Vector2D{T}) where T = Vector2D{T}(Tuple(A) .- Tuple(B))

Base. *(coeff::T, A::Vector2D{T}) where T = Vector2D{T}(coeff.*Tuple(A))

LinearAlgebra.norm(A::Vector2D) = norm(Tuple(A))

LinearAlgebra.dot(A::Vector2D{T}, B::Vector2D{T}) where T = dot(Tuple(A), Tuple(B))

Base. cos(A::Vector2D{T}, B::Vector2D{T}) where T = dot(A,B)/norm(A)/norm(B)

xdot(A::Vector2D{T}, B::Vector2D{T}) where T = A.x*B.y-A.y*B.x

Base. sin(A::Vector2D{T}, B::Vector2D{T}) where T = xdot(A,B)/norm(A)/norm(B)

Segment2D{T<:Real} = NamedTuple{(:A, :B), Tuple{Vector2D{T},Vector2D{T}}}

s=(A=(x=0,y=0), B=(x=1,y=2))

function intersect(s1::Segment2D{T},s2::Segment2D{T}) where T
    A = [s1.B[2]-s1.A[2]  s1.A[1]-s1.B[1]
         s2.B[2]-s2.A[2]  s2.A[1]-s2.B[1]]
    b = [s1.A[2]*(s1.A[1]-s1.B[1]) + s1.A[1]*(s1.B[2]-s1.A[2])
         s2.A[2]*(s2.A[1]-s2.B[1]) + s2.A[1]*(s2.B[2]-s2.A[2])]
    x,y = A\b
    if isinner((;x, y), s1)==false || isinner((;x, y), s2)==false
        return nothing
    end
    return Vector2D{T}((x,y))
end

isinner(P::Vector2D, s::Segment2D) = (s.A.x <= P.x <= s.B.x || s.A.x >= P.x >= s.B.x) && 
                                     (s.A.y <= P.y <= s.B.y || s.A.y >= P.y >= s.B.y)


s=(A=(x=-1.0,y=-1.0), B=(x=5.0,y=6.0))
d=(A=(x=2.0,y=-2.0), B=(x=-10.0,y=7.0))
println(intersect(s,d))
s = (A=(x=0.0, y=0.0), B=(x=1.0,y=2.0)) # сегмент
o = (A=(x=-1.0, y=-1.0), B=(x=3.0,y=3.0))
println(intersect(s,o))
println("THIS")
s=(A=(x=1.0,y=3.0), B=(x=5.0,y=1.0))
d=(A=(x=1.1,y=1.0), B=(x=9.0,y=7.0))
println(intersect(s,d))

println("THIS2")
s=(A=(x=5.0,y=1.0), B=(x=0.0,y=0.0))
d=(A=(x=7.0,y=0.4), B=(x=1.0,y=1.0))
println(intersect(s,d))

#- Вычисление угла между двумя направлениями на плоскости (заданными соответствующими векторами).
function angle_(s1::Segment2D{T},s2::Segment2D{T}) where T
    A = [s1.B[2]-s1.A[2] s1.B[1]-s1.A[1]]
    B = [s2.B[2]-s2.A[2] s2.B[1]-s2.A[1]]
    
    delta= abs(A[1]*B[1] + A[2]*B[2])/(sqrt(A[1]*A[1]+A[2]*A[2])*sqrt(B[1]*B[1]+B[2]*B[2]))

    return acos(delta) 
end
function angle_(A::Vector2D{T}, B::Vector2D{T}) where T
    if abs(A.x)==abs(B.x) && abs(A.y)==abs(B.y)
        return acos(dot(A, A)/abs(dot(A,A)))
    end
    return acos(dot(A, B)/(sqrt(dot(A,A)) * sqrt(dot(B,B))))
end 
function isinner(m::Vector2D{T}, plg::Vector{Vector2D{T}}) where T
    v1 = Vector2D{T}((plg[1][1]-m[1], plg[1][2]-m[2]))
    v2 =Vector2D{T}((plg[2][1]-m[1], plg[2][2]-m[2]))
    s = xdot(v1,v2)>0
    for t in 3:length(plg)
        v1 = v2
        v2 = Vector2D{T}((plg[t][1]-m[1], plg[t][2]-m[2]))
        if (xdot(v1, v2)>0)!=s
            return false    
        end
    end
    return true
end


s=(A=(x=-1.0,y=-1.0), B=(x=5.0,y=6.0))
d=(A=(x=-2.0,y=-2.0), B=(x=10.0,y=7.0))
println(angle_(s,d))


function f(X)
    return X[1]*cos(X[2]) - sin(X[1])*X[2]*X[2]
end
function CheckMap(f::Function,A,B)
    alfa=f(A)
    delta=f(B)
    if ((alfa>0 && delta>0) || (alfa<0 && delta<0))
        return true
    end
    return false
end

A=[-10 -10]
B=[-20 -20]
println(CheckMap(f,A,B))

A=[1 1]
B=[2 2]
println(CheckMap(f,A,B))


(x = -0.34782608695652173, y = -0.23913043478260868)
(x = 0.0, y = 0.0)
THIS
(x = 2.6482412060301503, y = 2.1758793969849246)
THIS2
(x = 3.666666666666667, y = 0.7333333333333334)
0.21866894587394162
false
true


In [87]:
Vector2D{T<:Real}=NamedTuple{(:x, :y), Tuple{T,T}}
function __double_ended!__(vertices::Vector{Vector2D{T}}) where T <: Real
    if vertices[begin]!=vertices[end]
        push!(vertices,vertices[end])
    end
    return vertices
end
abstract type AbstractPolygon{T<:Real} end
struct Polygon{T} <: AbstractPolygon{T}
    vertices::Vector{Vector2D{T}}
    Polygon{T}(vertices) where T = new(__double_ended!__(vertices))
end

A=Polygon{Float64}([Vector2D{Float64}((0.0,0.0)), Vector2D{Float64}((1.0,1.0)), Vector2D{Float64}((3.0,-1.0))])



Polygon{Float64}(NamedTuple{(:x, :y), Tuple{Float64, Float64}}[(x = 0.0, y = 0.0), (x = 1.0, y = 1.0), (x = 3.0, y = -1.0), (x = 3.0, y = -1.0)])

In [88]:

#- Проверка, является ли заданный плоский многоугольник выпуклым.
function CheckFigure(C::Polygon)
    n=length(C.vertices)
    n-=1
    for i in 1:n
        for j in 1:n
            if (i!=j)
                if (C.vertices[i][1]==C.vertices[j][1] && C.vertices[i][2]==C.vertices[j][2])
                    return false
                end
            end
        end
    end
    Q=[]
    for i in 1:n-1
        push!(Q,(A=(x=C.vertices[i][1],y=C.vertices[i][2]), B=(x=C.vertices[i+1][1],y=C.vertices[i+1][2])))
    end
    push!(Q,(A=(x=C.vertices[1][1],y=C.vertices[1][2]), B=(x=C.vertices[n][1],y=C.vertices[n][2])))
    for i in 1:length(Q)
        for j in 1:length(Q)
            if (i!=j)
                alfa=intersect(Q[i],Q[j])
                if (alfa==nothing || alfa==Q[i].A || alfa==Q[i].B ||  alfa==Q[j].A || alfa==Q[j].B)
                else
                    return false
                end
            end
        end
    end
    return true
end

C=Polygon{Float64}([Vector2D{Float64}((0.0,0.0)), Vector2D{Float64}((1.0,1.0)), Vector2D{Float64}((3.0,-1.0))])
println(CheckFigure(C))

true


In [89]:
struct ConvexPolygon{T} <: AbstractPolygon{T}
    vertices::Vector{Vector2D{T}}
    ConvexPolygon{T}(vertices) where T = new(__double_ended!__(vertices))
end
function isinner(A::Vector2D{T}, polygon::ConvexPolygon{T})::Bool where T
    sign_prev = sign(polygon[begin+1]-A, polygon[begin]-A)
    for i in firstindex(polygon.vertices)+1:lastindex(polygon.vertices)-1
        sign_cur += sign(polygon[i+1]-A, polygon[i]-A)
        if sign_prev * sign_cur < 0
            return false
        end 
    end
    return true
end

isinner (generic function with 4 methods)

In [106]:
function isinner(A::Vector2D{T}, polygon::AbstractPolygon{T})::Bool where T
    sum_angles = 0.0
    for i in firstindex(polygon.vertices):lastindex(polygon.vertices)-1
        sum_angles += angle(polygon[i+1]-A, polygon[i]-A)
    end
    if sum_angles < pi # фактически равно 0
        return true # точка внутри многоугольника
    else
        return false # точка снаружи
    end
end

isinner (generic function with 4 methods)

 Написать функцию, реализующую алгоритм Джарвиса построения выпуклой оболочки заданных точек плоскости .

In [91]:
function jarvis(points::Vector{Vector2D{T}})::ConvexPolygon{T} where T<:Real # алгоритм Джарвиса построения выпуклой оболочки заданного множества точек плоскости
    @assert length(points) > 1 # иначе операция в строке 9 будет не возможна
    # ydata = [points[i][2] for i in 1:length(points)] # - так создавался бы дополонительный массив
    ydata = (points[i][2] for i in 1:length(points))  # - а так массив не создается, вместо массива используется генератор
    i_start = findmin(ydata) # индекс самой нижней точки
    convex_shell = [i_start]
    ort_base = Vector2D{Int}((1,0)) # - этот вектор задает начальное базовое направление (горизонтално вправо)
    while next!(convex_shell, points, ort_base) != i_start
        ort_base = convex_shell[end] - convex_shell[end-1]  # - не нулевой вектор, задающий очередное базовое направление
    end
    return ConvexPolygon{T}(points[convex_shell]) # В конце и в начале массива convex_shell дважды содержится значение i_start 
end

function next!(convex_shell::Vector{Int64}, points::Vector{Vector2D{T}}, ort_base::Vector2D{T}) where T<:Real
    cos_max = typemin(T)
    i_base = convex_shell[end]
    resize!(convex_shell, length(convex_shell)+1)
    for i in eachindex(points)
        if points[i] == points[i_base] # тут не обязательно, что i == i_base
            continue
        end
        ort_i = points[i] - points[i_base] # - не нулевой вектор, задающий направление на очередную точку
        cos_i = cos(ort_base, ort_i)
        if cos_i > cos_max
            cos_max = cos_i
            convex_shell[end] = i 
        elseif cos_i == cos_max && quad_len(ort_i) > quad_len(ort_base) # на луче, содержащем сторону выпуклого многоугольника, может оказаться более двух точек заданного множества (надо выбрать самую дальнюю из них)
            convex_shell[end] = i
        end
    end
    return convex_shell[end]
end

quad_len(vec) = dot(vec, vec)

quad_len (generic function with 1 method)

 Написать функцию, реализующую алгоритм Грехома построения выпуклой оболочки заданных точек плоскости.


In [107]:
function grekhom!(points::Vector{Vector2D{T}})::ConvexPolygon{T} where T<:Real 
    ydata = (points[i][2] for i in 1:length(points))
    i_start = findmin(ydata) # индекс самой нижней точки
    points[begin], points[i_start] = points[i_start], points[begin]
    sort!(@view(points[begin+1:end]), by = (point -> angle(point, Vector2D{T}(1,0))))
    push!(points, points[begin]) # теперь points[end] == points[begin] 
    convex_polygon = [firstindex(points), firstindex(points)+1, firstindex(points)+2] # - в стек помещены первые 3 точки
    for i in firstindex(points)+3:lastindex(points)
        while sign(points[i]-points[convex_polygon[end]], points[convex_polygon[end-1]]-points[convex_polygon[end]]) < 0
            pop!(convex_polygon)
        end
        push!(convex_polygon, i)
    end
    pop!(points) # из конца массива извлечена предварительно продублированная первая точка (чтобы исходный массив не содержал лишних точек)
    return ConvexPolygon{T}(points[convex_polygon])  # convex_polygon[begin] == convex_polygon[end]
end

grekhom! (generic function with 1 method)

 Написать функцию, возвращающую значение ориентированной площади заданного плоского многоугольника, воспользовавшись методом трапеций.

In [93]:
A=Polygon{Float64}([Vector2D{Float64}((0.0,0.0)), Vector2D{Float64}((1.0,1.0)), Vector2D{Float64}((3.0,-1.0))])


function Stropec(A::Polygon)
    S=0
    n=length(A.vertices)
    for i in 1:n-1
        S+=((A.vertices[i][2]+A.vertices[i+1][2])/2)*(A.vertices[i+1][1]-A.vertices[i][1])
    end 
    S+=((A.vertices[n][2]+A.vertices[1][2])/2)*(A.vertices[1][1]-A.vertices[n][1])
    return abs(S)
end
function Stropec(plg::Vector{Vector2D{T}}) where T
    prev_p = plg[1]
    s = 0
    for i in 2:length(plg)
        p = plg[i]
        s += (prev_p.y+p.y)*(p.x-prev_p.x)/2
        prev_p = p
    end
    return abs(s)
end

A=Polygon{Float64}([Vector2D{Float64}((0.0,1.0)), Vector2D{Float64}((2.0,1.0)), Vector2D{Float64}((2.0,-1.0)), Vector2D{Float64}((0.0,-1.0))])
println(Stropec(A))

4.0


 Написать функцию, возвращающую значение ориентированной площади заданного плоского многоугольника, воспользовавшись методом треугольников .

In [94]:
function Streugolnik(C::Polygon)
    S=0
    n=length(C.vertices)
    for i in 1:n-1
        if (C.vertices[i][1]<C.vertices[i+1][1])
            k=1
        else
            k=-1
        end
        s=(A=(x=C.vertices[i][1], y=C.vertices[i][2]), B=(x=0.0,y=0.0))
        d=(A=(x=C.vertices[i+1][1], y=C.vertices[i+1][2]), B=(x=0.0,y=0.0))
        alfa=angle_(s,d)
        S+=sqrt((C.vertices[i][1])^2 + (C.vertices[i][2])^2)*sqrt((C.vertices[i+1][1])^2 + (C.vertices[i+1][2])^2)*sin(alfa)*k
    end 
    s=(A=(x=C.vertices[n][1], y=C.vertices[n][2]), B=(x=0.0,y=0.0))
    d=(A=(x=C.vertices[1][1], y=C.vertices[1][2]), B=(x=0.0,y=0.0))
    alfa=angle_(s,d)
    S-=sqrt((C.vertices[n][1])^2 + (C.vertices[n][2])^2)*sqrt((C.vertices[1][1])^2 + (C.vertices[1][2])^2)*sin(alfa)
    return abs(S)
end
C=Polygon{Float64}([Vector2D{Float64}((0.0,1.0)), Vector2D{Float64}((2.0,1.0)), Vector2D{Float64}((2.0,-1.0)), Vector2D{Float64}((0.0,-1.0))])
println(Streugolnik(C))

4.000000000000002


 Пусть имеется некоторый выпуклый многоугольник (выпуклая оболочка некотрого множества точек плоскости) и дана ещё дна тока. Требуется написать функцию, строющую выпуклую оболочку множества точек, включающее также и эту новую точку.

In [95]:
function add_point(points::Vector{Vector2D{T}}, point::Vector2D{T}) where T
    if isinner(point, points)
        return points
    end
    vector_point = Vector2D{T}((0,0))
    vector_nextp = Vector2D{T}((0, 0))
    vector_currentp = Vector2D{T}((points[1].x-points[end-1].x, points[1].y-points[end-1].y)) 
    for i in 1:length(points)-1
        vector_point = Vector2D{T}((point.x-points[i].x, point.y-points[i].y))
        vector_nextp = Vector2D{T}((points[i+1].x-points[i].x, points[i+1].y-points[i].y))
        if (angle_(vector_currentp, vector_nextp) >= angle_(vector_currentp, vector_point))
            k = i+1
            while true
                if k == length(points)
                    figure= points[:]
                    insert!(figure, k, point)
                    return figure
                end
                vector_point = Vector2D{T}((points[k].x-point.x, points[k].y-point.y))
                vector_mnext = Vector2D{T}((points[k+1].x-point.x, points[k+1].y-point.y))
                if angle_(vector_currentp, vector_point) >= angle_(vector_currentp, vector_mnext)
                    figure = points[begin:i]
                    push!(figure, point) #пересоздается оболчка, затраты памяти
                    append!(figure, points[k:end])
                    return figure
                end
                k+=1
            end
        end
        vector_currentp=vector_nextp
    end
end

add_point (generic function with 1 method)

 Реализовать построение выпуклой оболочки указанным выше способом.

In [96]:
function creat_figure(points::Vector{Vector2D{T}}) where T
    figure = points[1:3]
    push!(figure,points[1])
    for i in points[4:end]
        figure = add_point(figure, i)
    end
    return figure
end 

creat_figure (generic function with 1 method)

In [97]:
a=Vector2D{Float64}((0.0,0.0))
b=Vector2D{Float64}((4.5,1.5))
c=Vector2D{Float64}((6.5,4.5))
d=Vector2D{Float64}((0.0,6.4))
e=Vector2D{Float64}((3.5,7.5))
g=Vector2D{Float64}((6.0,6.0))
k=Vector2D{Float64}((35.5,37.5))
points = Vector{Vector2D{Float64}}([a,b,c,e,g,k])
creat_figure(points)

6-element Vector{NamedTuple{(:x, :y), Tuple{Float64, Float64}}}:
 (x = 0.0, y = 0.0)
 (x = 3.5, y = 7.5)
 (x = 35.5, y = 37.5)
 (x = 6.5, y = 4.5)
 (x = 6.0, y = 6.0)
 (x = 0.0, y = 0.0)

 Пусть в дополнение к условию задачи 6 известна площадь имеющейся выпуклой оболочки. Требуется аналогично задаче 7 реализовать вычисление индуктивной функции, возвращающей как саму выпуклую оболочку, так и ее площадь.

In [98]:
function add_point_with_S(points::Vector{Vector2D{T}}, point::Vector2D{T}, S::T) where T
    figure=add_point(points,point)
    return figure,Stropec(figure)
end

add_point_with_S (generic function with 1 method)

In [99]:
a=Vector2D{Float64}((0.0,0.0))
b=Vector2D{Float64}((4.5,1.5))
c=Vector2D{Float64}((6.5,4.5))
d=Vector2D{Float64}((0.0,6.4))
e=Vector2D{Float64}((3.5,7.5))
g=Vector2D{Float64}((6.0,6.0))
k=Vector2D{Float64}((35.5,37.5))
points = Vector{Vector2D{Float64}}([a,b,c,e,g])
figure=creat_figure(points)
S=Stropec(figure)
add_point_with_S(figure,k,S)



(NamedTuple{(:x, :y), Tuple{Float64, Float64}}[(x = 0.0, y = 0.0), (x = 3.5, y = 7.5), (x = 35.5, y = 37.5), (x = 6.5, y = 4.5), (x = 6.0, y = 6.0), (x = 0.0, y = 0.0)], 103.5)

 Написать функцию, проверяющую, явлется ли заданная последовательность точек (пердставленная типом Vector{Vector2D}), вершинами некоторого многоугольника (у многоугольника никакие две его стороны не пересекаются внутренним образом).

In [100]:

#- Проверка, является ли заданный плоский многоугольник выпуклым.
function CheckFigure(C::Vector{Vector2D{T}}) where T
    n=length(C)
    for i in 1:n
        for j in 1:n
            if (i!=j)
                if (C[i].x==C[j].x && C[i].y==C[j].y)
                    return false
                end
            end
        end
    end
    Q=[]
    for i in 1:n-1
        push!(Q,(A=(x=C[i].x,y=C[i].y), B=(x=C[i+1].x,y=C[i+1].y)))
    end
    push!(Q,(A=(x=C[1].x,y=C[1].y), B=(x=C[n].x,y=C[n].y)))
    for i in 1:length(Q)
        for j in 1:length(Q)
            if (i!=j)
                alfa=intersect(Q[i],Q[j])
                if (alfa==nothing || alfa==Q[i].A || alfa==Q[i].B ||  alfa==Q[j].A || alfa==Q[j].B)
                else
                    return false
                end
            end
        end
    end
    return true
end
a=Vector2D{Float64}((0.0,0.0))
b=Vector2D{Float64}((1.0,1.0))
c=Vector2D{Float64}((3.0,-1.0))
D = Vector{Vector2D{Float64}}([a,b,c])
println(CheckFigure(D))

true


 Написать функцию, вычисляющую выпуклую оболочку объединения некоторых двух множест точек плоскости, если заданы выпуклые оболочки каждого из них (сами множества считаются не заданными).

In [101]:
function creat_figure2(points1::Vector{Vector2D{T}},points2::Vector{Vector2D{T}}) where T
    figure = points1[1:3]
    push!(figure,points1[1])
    for i in points1[4:end]
        figure = add_point(figure, i)
    end
    for j in points2[1:end]
        figure = add_point(figure, j)
    end
    return figure
end 

creat_figure2 (generic function with 1 method)

In [102]:
a=Vector2D{Float64}((0.0,0.0))
b=Vector2D{Float64}((4.5,2.0))
c=Vector2D{Float64}((6.5,5.0))
d=Vector2D{Float64}((7.0,6.4))
e=Vector2D{Float64}((3.5,7.5))
g=Vector2D{Float64}((6.0,6.0))
k=Vector2D{Float64}((35.5,37.5))
l=Vector2D{Float64}((56.5,88.5))
points1 = Vector{Vector2D{Float64}}([a,b,c,d])
points2 = Vector{Vector2D{Float64}}([e,g,k,l])
creat_figure2(points1,points2)


6-element Vector{NamedTuple{(:x, :y), Tuple{Float64, Float64}}}:
 (x = 0.0, y = 0.0)
 (x = 3.5, y = 7.5)
 (x = 56.5, y = 88.5)
 (x = 35.5, y = 37.5)
 (x = 7.0, y = 6.4)
 (x = 0.0, y = 0.0)

 Написать функцию, вычисляющую выпуклую оболочку пересечения каких-либо двух множест точек плоскости, если заданы выпуклые оболочки каждого из них (сами множества считаются не заданными).

In [103]:
Base.angle(a::Vector2D{T}, b::Vector2D{T}) where T = atan(sin(a,b),cos(a,b))
function isinnerF(A::Vector2D{T}, polygon::Vector{Vector2D{T}})::Bool where T
    # проверка принадлежности точки внутренности многоугольника (не обязательно выпуклого)
        sum_angles = 0.0
        for i in firstindex(polygon):lastindex(polygon)-1
            sum_angles += angle(polygon[i+1]-A, polygon[i]-A)
        end
    
        if sum_angles < pi # фактически равно 0
            return true # точка внутри многоугольника
        else
            return false # точка снаружи
        end
end

isinnerF (generic function with 1 method)

In [104]:
function intersect(s1::Segment2D{T},s2::Segment2D{T}) where T
    A = [s1.B[2]-s1.A[2]  s1.A[1]-s1.B[1]
         s2.B[2]-s2.A[2]  s2.A[1]-s2.B[1]]
    b = [s1.A[2]*(s1.A[1]-s1.B[1]) + s1.A[1]*(s1.B[2]-s1.A[2])
         s2.A[2]*(s2.A[1]-s2.B[1]) + s2.A[1]*(s2.B[2]-s2.A[2])]
    x,y = A\b
    if isinner((;x, y), s1)==false || isinner((;x, y), s2)==false
        return nothing
    end
    return Vector2D{T}((x,y))
end

isinner(P::Vector2D, s::Segment2D) = (s.A.x <= P.x <= s.B.x || s.A.x >= P.x >= s.B.x) && 
                                     (s.A.y <= P.y <= s.B.y || s.A.y >= P.y >= s.B.y)

                                                            
function creat_figure3(points1::Vector{Vector2D{T}},points2::Vector{Vector2D{T}}) where T
    n=length(points1)
    m=length(points2)
    Q=[]
    Figure=Vector{Vector2D{Float64}}()
    for i in 1:n-1
        push!(Q,(A=(x=points1[i].x,y=points1[i].y), B=(x=points1[i+1].x,y=points1[i+1].y)))
    end
    push!(Q,(A=(x=points1[1].x,y=points1[1].y), B=(x=points1[n].x,y=points1[n].y)))
    for i in 1:m-1
        push!(Q,(A=(x=points2[i].x,y=points2[i].y), B=(x=points2[i+1].x,y=points2[i+1].y)))
    end
    push!(Q,(A=(x=points2[1].x,y=points2[1].y), B=(x=points2[m].x,y=points2[m].y)))
    for i in 1:length(Q)
        for j in 1:length(Q)
            if(i<j)
                alfa=intersect(Q[i],Q[j])
                if (alfa==nothing || alfa==Q[i].A || alfa==Q[i].B ||  alfa==Q[j].A || alfa==Q[j].B)
                else
                    push!(Figure,alfa)
                end
            end
        end
    end
    for i in 1:n
        if (!isinnerF(points1[i],points2))
            push!(Figure,points1[i])
        end
    end      
    for j in 1:m
        if (!isinnerF(points2[j],points1))
            push!(Figure,points2[j])
        end
    end     
    return (creat_figure(Figure))
end 
a=Vector2D{Float64}((0.0,0.0))
b=Vector2D{Float64}((0.5,2.0))
c=Vector2D{Float64}((5.0,1.0))
d=Vector2D{Float64}((1.1,1.1))
e=Vector2D{Float64}((1.4,1.5))
g=Vector2D{Float64}((2.0,1.0))
points1 = Vector{Vector2D{Float64}}([a,b,c])
points2 = Vector{Vector2D{Float64}}([d,e,g])
creat_figure3(points1,points2)

4-element Vector{NamedTuple{(:x, :y), Tuple{Float64, Float64}}}:
 (x = 1.1, y = 1.1)
 (x = 1.4, y = 1.5)
 (x = 2.0, y = 1.0)
 (x = 1.1, y = 1.1)

In [105]:
a=Vector2D{Float64}((0.0,0.0))
b=Vector2D{Float64}((1.0,3.0))
c=Vector2D{Float64}((5.0,1.0))
d=Vector2D{Float64}((1.0,1.0))
e=Vector2D{Float64}((9.0,7.0))
g=Vector2D{Float64}((7.0,0.4))
points3 = Vector{Vector2D{Float64}}([a,b,c])
points4 = Vector{Vector2D{Float64}}([d,e,g])
creat_figure3(points3,points4)

5-element Vector{NamedTuple{(:x, :y), Tuple{Float64, Float64}}}:
 (x = 2.6, y = 2.2)
 (x = 1.0, y = 1.0)
 (x = 3.6666666666666665, y = 0.7333333333333333)
 (x = 5.0, y = 1.0)
 (x = 2.6, y = 2.2)